<h1>Exploração dos dados com Apache Spark</h1>

<h3>Versão Spark utilizada <code>#!pip install pyspark==3.3.1</code></h3>

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession\
        .builder\
        .appName("twitter_transformation")\
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


25/01/05 19:21:56 WARN Utils: Your hostname, PcArthur resolves to a loopback address: 127.0.1.1; using 172.23.223.44 instead (on interface eth0)
25/01/05 19:21:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/05 19:21:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.json('../../datalake/twitter_datascience')

In [3]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_data|
+--------------------+--------------------+------------------+------------+
|[{62, 89, 2025-01...|{[{2025-01-03T19:...|{1234567890abcdef}|  2025-01-03|
|[{76, 48, 2025-01...|{[{2025-01-03T01:...|{1234567890abcdef}|  2025-01-03|
|[{26, 33, 2025-01...|{[{2025-01-03T23:...|{1234567890abcdef}|  2025-01-03|
|[{37, 2, 2025-01-...|{[{2025-01-03T20:...|              null|  2025-01-03|
|[{82, 62, 2024-12...|{[{2024-12-29T18:...|              null|  2024-12-29|
|[{27, 41, 2024-12...|{[{2024-12-30T16:...|              null|  2024-12-31|
|[{16, 91, 2024-12...|{[{2024-12-30T08:...|              null|  2024-12-30|
|[{55, 63, 2025-01...|{[{2025-01-04T14:...|              null|  2025-01-04|
+--------------------+--------------------+------------------+------------+



In [4]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [5]:
from pyspark.sql import functions as f

In [7]:
df.select(f.explode('data')).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [8]:
df.select(f.explode('data')).show()

+--------------------+
|                 col|
+--------------------+
|{62, 89, 2025-01-...|
|{45, 26, 2025-01-...|
|{43, 77, 2025-01-...|
|{40, 70, 2025-01-...|
|{72, 1, 2025-01-0...|
|{15, 17, 2025-01-...|
|{93, 84, 2025-01-...|
|{51, 65, 2025-01-...|
|{75, 59, 2025-01-...|
|{44, 76, 2025-01-...|
|{76, 48, 2025-01-...|
|{36, 89, 2025-01-...|
|{67, 74, 2025-01-...|
|{61, 25, 2025-01-...|
|{87, 69, 2025-01-...|
|{73, 100, 2025-01...|
|{9, 31, 2025-01-0...|
|{53, 32, 2025-01-...|
|{35, 54, 2025-01-...|
|{78, 99, 2025-01-...|
+--------------------+
only showing top 20 rows



In [9]:
df.select(f.explode('data').alias('tweets')).select("tweets.author_id", "tweets.conversation_id",
                                                    "tweets.created_at", "tweets.id",
                                                    "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [15]:
tweet_df = df.select(f.explode('data').alias('tweets')).select("tweets.author_id", "tweets.conversation_id",
                                                    "tweets.created_at", "tweets.id",
                                                    "tweets.public_metrics.*", "tweets.text")

In [16]:
tweet_df.show(5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       62|             89|2025-01-03T22:22:...| 91|        24|          0|         32|           11|Outro tweet fictí...|
|       45|             26|2025-01-03T02:44:...|  5|         5|         79|         41|            2|Tweet fictício ge...|
|       43|             77|2025-01-03T23:26:...| 49|        50|          0|         52|           36|Outro tweet fictí...|
|       40|             70|2025-01-03T21:53:...| 83|        92|         74|         63|           74|Outro tweet fictí...|
|       72|              1|2025-01-03T02:35:...| 89|        97|         51|         26|           20|Tweet fictício cr...|
+---------+-----

---

<h2>Criando dataframe de usuários</h2>

In [17]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [23]:
df.select(f.explode('includes.users')).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- username: string (nullable = true)



In [27]:
from pyspark.sql import functions as f 

user_df = df.select(f.explode('includes.users').alias('users')).select('users.*')

In [28]:
user_df.show(5)

+--------------------+---+------+--------+
|          created_at| id|  name|username|
+--------------------+---+------+--------+
|2025-01-03T19:33:...| 16|User 1|   user1|
|2025-01-03T10:28:...| 56|User 2|   user2|
|2025-01-03T12:17:...| 92|User 3|   user3|
|2025-01-03T15:46:...| 79|User 4|   user4|
|2025-01-03T11:50:...| 69|User 5|   user5|
+--------------------+---+------+--------+
only showing top 5 rows



In [29]:
tweet_df.coalesce(1).write.mode('overwrite').json('output/tweet')
user_df.coalesce(1).write.mode('overwrite').json('output/user')